In [1]:
# number of E and P elts in the universe
var('UN,UP')
UN=4; UP=2
UR=3

In [2]:
NalphaMat = matrix(3,UN+UP,
    [[ 1, 0,   0,  3,  0,   0],
     [ 0,  1,  0,  0,  5,   0],
     [ 0,  0,  7,  0,  0,   1]
    ])

NbetaMat = matrix(3,UN+UP,
    [[ 1, 0,   0,  11,  1,   0],
     [ 0, 1,  13,   0,  0,   0],
     [ 0, 1,   0,   0,  17,  1]
    ])




In [3]:
Crg = PolynomialRing(RationalField(),
                     ['r%s'%i for i in range(1, UN+1)] + ['g%s'%i for i in range(1, UN+1)]
                    + ['U','V','W','Y'])
Crg.inject_variables()


#CrgF=Crg.fraction_field()
#CrgF.inject_variables()
#THIS FAILS WHEN WE TRY TO MULTIPLY AN EXTERIOR ALG ELE BY A POLY. GENERATOR

Ext=\
    ExteriorAlgebra(Crg,\
        ['p%s'%i  for i in range(1, UP+1)]+\
        ['pa%s'%i for i in range(1, UP+1)]+\
        ['pb%s'%i for i in range(1, UP+1)]+\
        ['e%s'%i  for i in range(1, UN+1)])
Ext.inject_variables() 

Rrrs=[Crg.gen(i)    for i in range(0, UN)]
Gees=[Crg.gen(i)    for i in range(UN, 2*UN)]
Pees=[Ext.gen(i)    for i in range(0, UP)]
Paas=[Ext.gen(i)    for i in range(UP, 2*UP)]
Pbes=[Ext.gen(i)    for i in range(2*UP, 3*UP)]
Eees=[Ext.gen(i)    for i in range(3*UP, 3*UP+UN)]

NSeq=Pees+Eees
PEVol=prod(NSeq)
EVol=prod(Eees)

def RofE(e):
    return(Rrrs[Eees.index(e)])
def GofE(e):
    return(Gees[Eees.index(e)])

diagRrrs=diagonal_matrix(Rrrs)
diagGees=diagonal_matrix(Gees)

matII=block_matrix([[matrix(UP)         ,matrix(UP)    ,matrix(UP)   ,matrix(UP,UN)],
                    [identity_matrix(UP),matrix(UP)    ,matrix(UP)   ,matrix(UP,UN)],       
                    [matrix(UP)         ,matrix(UP)    ,matrix(UP)   ,matrix(UP,UN)],
                    [matrix(UN,UP)      ,matrix(UN,UP) ,matrix(UN,UP),diagGees     ]
                   ])
matVV=block_matrix([[matrix(UP)         ,matrix(UP)    ,matrix(UP)   ,matrix(UP,UN)],
                    [matrix(UP)         ,matrix(UP)    ,matrix(UP)   ,matrix(UP,UN)],       
                    [identity_matrix(UP),matrix(UP)    ,matrix(UP)   ,matrix(UP,UN)],
                    [matrix(UN,UP)      ,matrix(UN,UP) ,matrix(UN,UP),diagRrrs     ]
                   ])

II=Ext.lift_module_morphism(matII)
VV=Ext.lift_module_morphism(matVV)


Defining r1, r2, r3, r4, g1, g2, g3, g4, U, V, W, Y
Defining p1, p2, pa1, pa2, pb1, pb2, e1, e2, e3, e4


In [4]:
#Exterior Algebra operations only
#Exterior Algebra with ground set operations are defined later.


def contract(X,E):
    #only contract by monomials
    #Is used to implement LvSE and LhSE contraction where the ground set is Palpha+Pbeta+E
    assert (E.length()==1)
    return (X.interior_product(E)*(-1)^(E.degree()*(X.degree()-E.degree())))

def delete(X,e):
    #the subtraction formula only works for vectors.
    assert (e.degree()==1 & e.length()==1)
    return (X - contract(X,e)*e)

def delete_contract(X,e):
    #only contract by monomials
    assert (e.degree()==1 & e.length()==1)
    contr = X.interior_product(e)*(-1)^(e.degree()*(X.degree()-e.degree()))
    dele = X - contr*e
    return (dele, contr)

def perp(VOL,N):
    return (VOL.interior_product(N))

In [5]:
#Exterior algebra elements with ground sequenced ground set
class ExtS:
    def __init__(this, extN, seqS):
        this.extN = extN
        this.seqS = copy(seqS)
        this._set_VOL()      
    
    def contraction(this,e):
        #does only one
        assert (e.length()==1 & e.degree()==1)
        result = this._copy_remove(e)
        result.extN = contract(result.extN,e)
        return (result)
    def deletion(this,e):
        #does only one
        assert (e.length()==1 & e.degree()==1)  
        result = this._copy_remove(e)
        result.extN = delete(result.extN,e)
        return (result)
    def perp(this):
        result=this._copy()
        result.extN = perp(this.VOL, this.extN)
        return (result)
    
    def printme(this):
        print("element:",this.extN)
        print("set orientation:",this.seqS)
        print("Volume:",this.VOL)
        
    def _set_VOL(this):
        acc = 1
        for v in this.seqS:
            acc = acc*v
        this.VOL = acc
    def _copy(this):
        return ExtS(this.extN, this.seqS)    
    def _copy_remove(this,e):
        result = this._copy()
        result.seqS.remove(e)
        result._set_VOL()
        return(result)



In [6]:
def ExtFromMatrix(NMat):
    ExtCol=NMat*(matrix([NSeq]).transpose())
    ExtRes=1
    for i in range(0,ExtCol.nrows()):
        ExtRes=ExtRes*ExtCol[i,0]
    return ExtRes

def ExtSFromExt(extN):
    seq=list(extN.parent().gens()[0:UP])+list(extN.parent().gens()[3*UP:])
    return (ExtS(extN, seq))

def ExtSFromMatrix(Matr):
    extN=ExtFromMatrix(Matr)
    return (ExtSFromExt(extN))

ExtSNalpha=ExtSFromMatrix(NalphaMat)
ExtSNbeta =ExtSFromMatrix(NbetaMat)

In [7]:
def LvS(ExtStop,ExtSbot):
    assert ((ExtStop.extN.parent()==ExtSbot.extN.parent()) & 
            (ExtStop.seqS==ExtSbot.seqS))
    newSeq=list(ExtStop.extN.parent().gens()[UP:3*UP])+ExtStop.seqS[UP:]
    newExt=II(ExtStop.extN)*VV(ExtSbot.extN)
    return ExtS(newExt,newSeq)
def LvSE(ExtStop,ExtSbot):
    LvSret=LvS(ExtStop,ExtSbot)
    ESeq=list(ExtStop.seqS[UP:])
    PaPbSeq=list(ExtStop.extN.parent().gens()[UP:3*UP])
    EE=1
    for v in ESeq:
        EE=EE*v
    newExt=contract(LvSret.extN,EE)
    return( ExtS(newExt,PaPbSeq))
def LhS(Extleft,Extright):
    return LvS(Extleft,Extright.perp())
def LhSE(Extleft,Extright):
    return LvSE(Extleft,Extright.perp())

In [8]:
def nstr(n):
    st=""
    for l in ["%s"%i for i in range(1,n+1)]:
        st=st+l
    return st


class Tree():
    autopick = True
    debug = False
    def __init__(self,seq,Nalpha,Nbeta,nameAlpha="NAlpha",nameBeta="NBeta",parent=None):
        self.Nalpha=Nalpha
        self.Nbeta =Nbeta
        self.nameAlpha=nameAlpha
        self.nameBeta=nameBeta
        self.seq = seq
        self.parent = parent
        if not Tree.autopick or Tree.debug:
            print("\nWorking on:")
            print(nameAlpha+":")
            self.Nalpha.printme()
            print(nameBeta+":")
            self.Nbeta.printme()
        if len(seq) != 0:
            if len(seq) == 1:
                self.e = seq[0]
            else:
                if Tree.autopick:
                    self.e = seq[0]
                else:
                    print("\n",seq)
                    self.e = eval(input("Pick one:"))
            #print(self.e)
            nseq=copy(seq)
            nseq.remove(self.e)
            if Tree.debug: print("Make contr subtree:") 
            estr=self.e.__repr__() 
            self.contrchild=Tree(nseq,self.Nalpha.contraction(self.e),self.Nbeta.contraction(self.e),
                                 self.nameAlpha+"/"+estr,self.nameBeta+"/"+estr, self)
            if Tree.debug: print("\nMake delet subtree:")
            self.deletchild=Tree(nseq,Nalpha.deletion(self.e),Nbeta.deletion(self.e),
                                 self.nameAlpha+"\\"+estr, self.nameBeta+"\\"+estr, self)
            self.TF=GofE(self.e)*self.contrchild.TF+RofE(self.e)*self.deletchild.TF
        else:
            self.contrchild = None
            self.deletchild = None
            if Tree.debug: self.Nalpha.printme()
            if Tree.debug: self.Nbeta.printme()
            self.TF=II(self.Nalpha.extN)*VV(perp(self.Nbeta.VOL,self.Nbeta.extN))
    
    def printnode(self):
        print(self.nameAlpha)
        self.Nalpha.printme()
        print(self.nameBeta)
        self.Nbeta.printme()
        print(self.seq)
        if self.contrchild:            
            print(self.contrchild.nameAlpha,self.contrchild.nameBeta)
        else:
            print("No contraction child.")
        if self.deletchild:
            print(self.deletchild.nameAlpha,self.deletchild.nameBeta)
        else:
            print("No contraction child.")
        print(self.TF)

    def printme(self,depth=0):
        print('\n'+(nstr(depth))+"Node:",self.seq)
        print(('--'*depth)+self.nameAlpha)
        self.Nalpha.printme()
        print(('--'*depth)+self.nameBeta)
        self.Nbeta.printme()
        if(self.contrchild):
            self.contrchild.printme(depth+1)
        if(self.deletchild):
            self.deletchild.printme(depth+1)
            
        

In [23]:


class TreeUVWY():
    autopick = True
    debug = False
    def __init__(self,seq,Nalpha,Nbeta,
                 nameAlpha="NAlpha",nameBeta="NBeta",parent=None):

        self.Nalpha=Nalpha 
        self.Nbeta =Nbeta
        self.nameAlpha=nameAlpha
        self.nameBeta=nameBeta
        self.seq = seq
        self.parent = parent
        if (not TreeUVWY.autopick) or TreeUVWY.debug:
            print("\nWorking on:")
            print(nameAlpha+":")
            self.Nalpha.printme()
            print(nameBeta+":")
            self.Nbeta.printme()
        
        self.leftchild = None
        self.rightchild = None
        
        if (self.Nalpha.extN == 0 and self.Nbeta.extN == 0):
            if TreeUVWY.debug: self.Nalpha.printme()
            if TreeUVWY.debug: self.Nbeta.printme()
            self.TF=0
            return
        if len(seq) != 0:
            if len(seq) == 1:
                self.e = seq[0]
            else:
                if TreeUVWY.autopick:
                    self.e = seq[0]
                else:
                    print("\n",seq)
                    self.e = eval(input("Pick one:"))
            #print(self.e)
            nseq=copy(seq)
            nseq.remove(self.e)
            estr=self.e.__repr__()
            CA=Nalpha.contraction(self.e)
            CB=self.Nbeta.contraction(self.e)
            DA=Nalpha.deletion(self.e)
            DB=Nbeta.deletion(self.e)
            
            Lcoef=0
            Rcoef=0
            
            if (CA.extN != 0 and CB.extN != 0):
                self.leftchild=TreeUVWY(nseq, CA, CB, self.nameAlpha+"/"+estr,self.nameBeta+"/"+estr, self)
                Lcoef=GofE(self.e)
            if (CA.extN != 0 and CB.extN == 0):
                self.leftchild=TreeUVWY(nseq, CA, DB, self.nameAlpha+"/"+estr,self.nameBeta+"\\"+estr, self)
                Lcoef=V*GofE(self.e)
            if (CA.extN == 0 and CB.extN != 0):
                self.leftchild=TreeUVWY(nseq, DA, CB, self.nameAlpha+"\\"+estr,self.nameBeta+"/"+estr, self)
                Lcoef=U*GofE(self.e)
            if (CA.extN == 0 and CB.extN == 0):
                self.leftchild=TreeUVWY(nseq, DA, DB, self.nameAlpha+"\\"+estr,self.nameBeta+"\\"+estr, self)
                Lcoef=U*V*GofE(self.e)
                
            if (DA.extN != 0 and DB.extN != 0):
                self.rightchild=TreeUVWY(nseq, DA, DB, self.nameAlpha+"\\"+estr,self.nameBeta+"\\"+estr,self)
                Rcoef=RofE(self.e)
            if (DA.extN != 0 and DB.extN == 0):
                self.rightchild=TreeUVWY(nseq, DA, CB, self.nameAlpha+"\\"+estr,self.nameBeta+"/"+estr,self)
                Rcoef=Y*RofE(self.e)
            if (DA.extN == 0 and DB.extN != 0):
                self.rightchild=TreeUVWY(nseq, CA, DB, self.nameAlpha+"/"+estr,self.nameBeta+"\\"+estr,self)
                Rcoef=W*RofE(self.e)
            if (DA.extN == 0 and DB.extN == 0):
                self.rightchild=TreeUVWY(nseq, CA, CB, self.nameAlpha+"/"+estr,self.nameBeta+"/"+estr,self)
                Rcoef=W*Y*RofE(self.e)
            
            self.TF=Lcoef*self.leftchild.TF+Rcoef*self.rightchild.TF
        else:
            self.contrchild = None
            self.deletchild = None
            if Tree.debug: self.Nalpha.printme()
            if Tree.debug: self.Nbeta.printme()
            self.TF=II(self.Nalpha.extN)*VV(perp(self.Nbeta.VOL,self.Nbeta.extN))
    
    def printnode(self):
        print(self.nameAlpha)
        self.Nalpha.printme()
        print(self.nameBeta)
        self.Nbeta.printme()
        print(self.seq)
        
        print("\n",self.TF.list())
        print("\n",self.TF)
        print("\n")
        
        if self.leftchild:            
            print(self.leftchild.nameAlpha,self.leftchild.nameBeta)
        else:
            print("No contraction child.")
        if self.rightchild:
            print(self.rightchild.nameAlpha,self.rightchild.nameBeta)
        else:
            print("No contraction child.")
        print(self.TF)

    def printme(self,depth=0):
        print('\n'+(nstr(depth))+"Node:",self.seq)
        print(('--'*depth)+self.nameAlpha)
        self.Nalpha.printme()
        print(('--'*depth)+self.nameBeta)
        self.Nbeta.printme()
        
        print("\n",self.TF.list())
        print("\n",self.TF)
        print("\n")
        
        if(self.leftchild):
            self.leftchild.printme(depth+1)
        if(self.rightchild):
            self.rightchild.printme(depth+1)
            
        

In [24]:

fooAuto=TreeUVWY([e1,e2,e3,e4],ExtSNalpha,ExtSNbeta)



In [19]:
TreeUVWY.autopick=False
fooMan=TreeUVWY([e1,e2,e3,e4],ExtSNalpha,ExtSNbeta)


 [e1, e2, e3, e4]
Pick one:e3

 [e1, e2, e4]
Pick one:e1

 [e2, e4]
Pick one:e4

 [e2, e4]
Pick one:e4

 [e1, e2, e4]
Pick one:e1

 [e2, e4]
Pick one:e4

 [e2, e4]
Pick one:e2


In [20]:
fooMan.TF.list()

[((2, 3, 5), -91*r2*r3*g1*g4*U),
 ((2, 4, 5), 455*r2*g1*g3*g4*U),
 ((3, 4, 5), 3003*r3*g1*g2*g4*U),
 ((2, 3), r1*r2*r3*r4*W*Y - 91*r2*r3*r4*g1 + r1*r2*r3*g4),
 ((2, 4), -455*r2*r4*g1*g3 + 5*r1*r2*g3*g4),
 ((2, 5), -7735*r2*r4*g1*g3),
 ((3, 4), -33*r1*r3*r4*g2*W*Y + 3003*r3*r4*g1*g2 - 33*r1*r3*g2*g4),
 ((4, 5), 1365*g1*g2*g3*g4*U*V - 255255*r4*g1*g2*g3),
 ((2,), -85*r1*r2*r4*g3*W),
 ((4,), 15*r1*g2*g3*g4*V - 2805*r1*r4*g2*g3*W)]

In [16]:
fooAuto.TF.list()

[((2, 3, 5), -91*r2*r3*g1*g4*U),
 ((2, 4, 5), 455*r2*g1*g3*g4*U),
 ((3, 4, 5), -3003*r3*g1*g2*g4*U),
 ((2, 3), r1*r2*r3*r4*W*Y - 91*r2*r3*r4*g1 + r1*r2*r3*g4),
 ((2, 4), -455*r2*r4*g1*g3 + 5*r1*r2*g3*g4),
 ((2, 5), -7735*r2*r4*g1*g3),
 ((3, 4), -33*r1*r3*r4*g2*W*Y + 3003*r3*r4*g1*g2 - 33*r1*r3*g2*g4),
 ((4, 5), -255255*g1*g2*g3*g4*U*V - 255255*r4*g1*g2*g3),
 ((2,), -85*r1*r2*r4*g3*W),
 ((4,), 2805*r1*g2*g3*g4*V + 2805*r1*r4*g2*g3*W)]

In [25]:
TreeUVWY.autopick=False
fooMan=TreeUVWY([e1,e2,e3,e4],ExtSNalpha,ExtSNbeta)


Working on:
NAlpha:
element: 7*p1*p2*e1 + p1*p2*e4 - 35*p1*e1*e3 + 5*p1*e3*e4 + 21*p2*e1*e2 - 3*p2*e2*e4 + 105*e1*e2*e3 + 15*e2*e3*e4
set orientation: [p1, p2, e1, e2, e3, e4]
Volume: p1*p2*e1*e2*e3*e4
NBeta:
element: -13*p1*p2*e1 + 17*p1*p2*e3 + p1*p2*e4 + 221*p1*e1*e3 + 13*p1*e1*e4 - 143*p2*e1*e2 - 13*p2*e1*e3 - 187*p2*e2*e3 - 11*p2*e2*e4 - p2*e3*e4 - 2431*e1*e2*e3 - 143*e1*e2*e4 - 13*e1*e3*e4
set orientation: [p1, p2, e1, e2, e3, e4]
Volume: p1*p2*e1*e2*e3*e4

 [e1, e2, e3, e4]
Pick one:e3

Working on:
NAlpha/e3:
element: -35*p1*e1 - 5*p1*e4 + 105*e1*e2 - 15*e2*e4
set orientation: [p1, p2, e1, e2, e4]
Volume: p1*p2*e1*e2*e4
NBeta/e3:
element: 17*p1*p2 + 221*p1*e1 - 13*p2*e1 - 187*p2*e2 + p2*e4 - 2431*e1*e2 + 13*e1*e4
set orientation: [p1, p2, e1, e2, e4]
Volume: p1*p2*e1*e2*e4

 [e1, e2, e4]
Pick one:e1

Working on:
NAlpha/e3/e1:
element: -35*p1 - 105*e2
set orientation: [p1, p2, e2, e4]
Volume: p1*p2*e2*e4
NBeta/e3/e1:
element: 221*p1 - 13*p2 + 2431*e2 - 13*e4
set orientation: [p1

In [28]:
(fooMan.TF-fooAuto.TF).list()

[((3, 4, 5), 6006*r3*g1*g2*g4*U),
 ((4,), -2790*r1*g2*g3*g4*V - 5610*r1*r4*g2*g3*W)]

In [27]:
fooAuto.TF.list()

[((2, 3, 5), -91*r2*r3*g1*g4*U),
 ((2, 4, 5), 455*r2*g1*g3*g4*U),
 ((3, 4, 5), -3003*r3*g1*g2*g4*U),
 ((2, 3), r1*r2*r3*r4*W*Y - 91*r2*r3*r4*g1 + r1*r2*r3*g4),
 ((2, 4), -455*r2*r4*g1*g3 + 5*r1*r2*g3*g4),
 ((2, 5), -7735*r2*r4*g1*g3),
 ((3, 4), -33*r1*r3*r4*g2*W*Y + 3003*r3*r4*g1*g2 - 33*r1*r3*g2*g4),
 ((4, 5), -255255*g1*g2*g3*g4*U*V - 255255*r4*g1*g2*g3),
 ((2,), -85*r1*r2*r4*g3*W),
 ((4,), 2805*r1*g2*g3*g4*V + 2805*r1*r4*g2*g3*W)]

In [ ]:
HAPPYMAYBE=LvSE(ExtSNalpha,ExtSNbeta.perp())
HAPPYMAYBE.printme()
HAPPYTWO=LhSE(ExtSNalpha,ExtSNbeta)
HAPPYTWO.printme()
HAPPYMAYBE.extN==HAPPYTWO.extN

In [ ]:
foo.printme()


In [ ]:
foo.printnode()

In [ ]:
fooMan.printnode()

In [ ]:
fooAuto.printnode()

In [ ]:
fooAuto.TF.list()

In [ ]:
fooMan.TF.list()

In [ ]:
ExtSNalpha.printme()

In [ ]:
100-int(random()*50)

In [ ]:
matrix(3,UN+UP,[100-int(random()*50) for i in range(0,18)])

In [ ]:
UR=3

AM = matrix(3,UN+UP,[100-int(random()*50) for i in range(0,18)])
BM =matrix(3,UN+UP,[100-int(random()*50) for i in range(0,18)])

print(AM)
print(BM)

In [ ]:
EAM=ExtSFromMatrix(AM)
EBM=ExtSFromMatrix(BM)
EAM.printme()
EBM.printme()

In [ ]:
bar=TreeUVWY([e1,e2,e3,e4],EAM,EBM,"AL","BE")

In [ ]:
bar.printme()

In [ ]:
for a in HAPPYTWO.extN.list():
    print(a)

In [ ]:
def TestDeleContIdentity(ExtSNalpha,ExtSNbeta,e):
    start=LhSE(ExtSNalpha,ExtSNbeta)
    contr_alpha=ExtSNalpha.contraction(e)
    contr_beta =ExtSNbeta.contraction(e)
    delet_alpha=ExtSNalpha.deletion(e)
    delet_beta =ExtSNbeta.deletion(e)
    contrLE    =LhSE(contr_alpha,contr_beta)
    deletLE    =LhSE(delet_alpha,delet_beta)
    contrExt   =contrLE.extN
    deletExt   =deletLE.extN
    i=ExtSNalpha.seqS.index(e)-UP
    ge=Gees[i]
    re=Rrrs[i]
    for a in start.extN.list():
        print(a)
    print()
    for a in (ge*contrExt).list():
        print(a)
    print()
    for a in (re*deletExt).list():
        print(a)
    
    print("\n",start.extN==ge*contrExt+re*deletExt)

In [ ]:
TestDeleContIdentity(ExtSNalpha,ExtSNbeta,e4)

In [ ]:
n=UN
p=UP

var('r cr')

r = 3

cr = n+p-r

In [ ]:
#these will be overwritten when they are built from a graph below

Nal =   matrix(Crg,  r,  n+p)
NbePe = matrix(Crg, cr,  n+p)


In [ ]:
#
# Supply a graph
#
Graph1MyEdges= [(2, 1, p1),(4, 3, p2),(1, 3, e1), (2, 3, e2), (4, 1, e3),(2, 4, e4)  ]
#HA I can label the edges with Ext generators!

Graph1=DiGraph([[1,2,3,4],Graph1MyEdges])

pos_dict = {
 1: [.33, .25],
 2: [.33, 0],
 3: [.6, .33],
 4: [0.6, -.1]}

myplot=Graph1.plot(edge_labels=True,vertex_labels=True,pos=pos_dict,save_pos=True)
myplot.save("K4.pdf")
myplot

In [ ]:
#Graph1.edges()

NalFull=Graph1.incidence_matrix(oriented=True, edges=Graph1MyEdges)
                       
NalFull

In [ ]:
temp=[i for i in range(r,Graph1.num_verts())]
Nal=NalFull.delete_rows(temp)
assert((Nal.nrows()==r) & (Nal.ncols()==n+p))
Nal

In [ ]:
latex(Nal)

In [ ]:
NbePe=Nal.right_kernel().basis_matrix()
NbePe

In [ ]:
latex(NbePe)

In [ ]:
NalG=   Nal  *diagonal_matrix([1 for i in range(0,p)]+Gees)
NbePeR= NbePe*diagonal_matrix([1 for i in range(0,p)]+Rrrs)

In [ ]:
latex(NalG)

In [ ]:
NbePeR

In [ ]:
latex(NbePeR)

In [ ]:
Lal=NalG   * (matrix([Paas+Eees]).transpose());print("Lal:\n",Lal)
Lbe=NbePeR * (matrix([Pbes+Eees]).transpose());print("\nLbe:\n",Lbe)

In [ ]:
dictLal=Lal.dict();dictLal

In [ ]:
dictLbe=Lbe.dict();dictLbe

In [ ]:
ResAl=1
for i in range(0,r):
    ResAl=ResAl*(dictLal.get((i,0)))
print("ResAl\n")
print(ResAl)
ResBe=1
for i in range(0,cr):
    ResBe=ResBe*(dictLbe.get((i,0)))
print("\nResBe\n")
print(ResBe)
Res=ResAl*ResBe

LLL=Res

#print("final result")
#print(Res)
Reslist=Res.list()
print("\n Res list:\n")
Reslist


In [ ]:
RESULT=0
for thing in Reslist:
    print(tuple(range(3*p,3*p+n)))
    print("\nTry:",thing[0])
    if thing[0][p:p+n]==tuple(range(3*p,3*p+n)):
        print(" \n", thing)
        prgens=1
        print("\n",thing[0][0:p])
        for i in thing[0][0:p]:
            prgens=prgens*(Ext.gens()[i])
        print(" \nMonomial:", prgens)
        print("Coef:",thing[1])
        term = thing[1]*prgens
#        print(" \n", term)
        RESULT = RESULT + term
#print("\nHERE IT IS:", RESULT)

In [ ]:
RESULT.list()

In [ ]:
RESULT==GREAT

In [ ]:
HAPPYMAYBE.extN==RESULT

In [ ]:
TEST==RESULT

In [ ]:
LLL

In [ ]:
LLL

In [ ]:
delete(LLL,e1)

In [ ]:
contract(LLL,e1)


In [ ]:
delete_contract(LLL,e1)